In [ ]:
import pandas as pd

In [ ]:
import mysql.connector as msql
from mysql.connector import Error
dataset = []
try:
    conn = msql.connect(host='localhost', database='DMHW1', user='root', password='TP6XU4JO3@@')
    if conn.is_connected():
        mycursor = conn.cursor()
        mycursor.execute("select database();")
        record = mycursor.fetchone()
        print("You're connected to database: ", record)
        mycursor.execute(""" select
                                d.customer_id,
                                pc.product_category,
                                d.state_province, 
                                d.yearly_income, 
                                d.gender,
                                CONCAT('totally ',d.total_children,' children') as 'total_children',
                                CONCAT('own ', d.num_cars_owned ,' cars') as 'owned_cars',
                                CONCAT('totally ', d.num_children_at_home,' children at home') as 'children_at_home',
                                d.education, 
                                d.occupation, 
                                CASE WHEN d.houseowner = 'Y' then 'Is houseowner'
                                else 'Is not houseowner'
                                end as 'houseowner'
                                from DMHW1.sales_fact_1998 a 
                                join DMHW1.product c 
                                on a.product_id = c.product_id 
                                join DMHW1.product_class pc
                                on c.product_class_id = pc.product_class_id
                                join DMHW1.customer d 
                                on a.customer_id = d.customer_id 
                                union 
                                select 
                                d.customer_id,
                                pc.product_category,
                                d.state_province,
                                d.yearly_income, 
                                d.gender, 
                                CONCAT('totally ',d.total_children,' children') as 'total_children',
                                CONCAT('own ', d.num_cars_owned ,' cars') as 'owned_cars',
                                CONCAT('totally ', d.num_children_at_home,' children at home') as 'children_at_home',	
                                d.education, 
                                d.occupation, 
                                CASE WHEN d.houseowner = 'Y' then 'Is houseowner'
                                else 'Is not houseowner'
                                end as 'houseowner'
                                from DMHW1.sales_fact_dec_1998 a 
                                join DMHW1.product c 
                                on a.product_id = c.product_id 
                                join DMHW1.product_class pc
                                on c.product_class_id = pc.product_class_id
                                join DMHW1.customer d 
                                on a.customer_id = d.customer_id  limit 60000
                         """)
        print("data is fetched....")
        #loop through the data fetched
        results = mycursor.fetchall()
        a = dict()
        for item in results:
            row = list(item)
            dicKey = ""
            for i in range(len(row)):
                if type(row[i]) is not str:
                    row[i] = str(row[i])
                if i == 0:
                    dicKey = row[i]
                    if dicKey not in a.keys():
                        a[dicKey] = []
                elif i != 0:
                    if dicKey in a.keys(): 
                        ls = a[dicKey]
                        ls.append(row[i])
                        a[dicKey] = ls
            #print(a[dicKey])
        for key,value in a.items():
            dataset.append(value)
except Error as e:
            print("Error while connecting to MySQL", e)

In [ ]:
#from apyori import apriori
from mlxtend.frequent_patterns import fpgrowth
from mlxtend.preprocessing import TransactionEncoder
te = TransactionEncoder()
te_ary = te.fit(dataset).transform(dataset)
df = pd.DataFrame(te_ary, columns=te.columns_)

In [ ]:
frequent_sets = fpgrowth(df, min_support=0.01,use_colnames=True) 


In [ ]:
from mlxtend.frequent_patterns import association_rules
rules = association_rules(frequent_sets, metric='confidence', min_threshold=0.8)
rules["antecedent_len"] = rules["antecedents"].apply(lambda x:len(x))
rules["consequent_len"] = rules["consequents"].apply(lambda x:len(x))
rules[(rules["antecedent_len"] >= 1)& (rules["consequent_len"] >=1)].sort_values(by=['confidence'], ascending=False).to_csv('quantitive_rules_3.csv')
